In [62]:
%reset -f
%load_ext autoreload
%autoreload 2

import Odin as backend

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
# validation test for scalar diffusion with a constant source term
# this validation test follows "H K Versteeg and W Malalasekera, An Introduction to Computational Fluid Dynamics
# THE FINITE VOLUME METHOD, Second Edition, 2007" examples 4.1,4.2

# the temperature along the x-axis should develop as:
def yFunc(x):
    return ((T_r-T_l)/LenX + S/(2*k)*(LenX-x))*x + T_l

# write '4.1' or '4.2'
exampleId = '4.2'

# geometric parameters
LenX = {'4.1':0.5, '4.2':0.02}[exampleId]
LenY = {'4.1':0.1, '4.2':0.004}[exampleId]

# thermal conductivity [ W / (m K) ]
k = {'4.1':1000, '4.2':0.5}[exampleId]

# heat source [ W / m^3 ]
R = {'4.1':0, '4.2':1e6}[exampleId]
S = R

# const temperature at left/right boundaries [K]
T_l = {'4.1':100, '4.2':100}[exampleId]
T_r = {'4.1':500, '4.2':200}[exampleId]

resolution = {'4.1':10, '4.2':2500}[exampleId]


# the cross sectional area is hard coded in createMatrixCoefficients()
# the cross sectional area should be defined by the geometry/mesh

In [64]:
geom = backend.createGeometry( 'rectangle', [LenX, LenY] )
mesh = backend.createMesh( geom, res=resolution )

#mesh.getStats()

# example: incompressible flow but with temp-dep density, i.e. BoussinesqFlow
# mulitphase stokes flow
myFlowModels = {
    #'flowField'     :   backend.FlowModels.IncompressibleFlow(),
    #'soluteTransport':  backend.FlowModels.ScalarConvectionDiffusion(),
    'temperature'   :   backend.FlowModels.ScalarConvectionDiffusion(scalarFieldName='T', velocityFieldName='U', diffusionCoefficientName='k', sourceFieldName='R')
    }
myFlowModels['temperature']._properties['diffusionCoefficient'] = k
myFlowModels['temperature'].showContinuumProperties()

sim1 = backend.Simulation(myFlowModels, mesh, geom )
#print(type(sim1._variableFields['U']).__name__) # i can use this to remove the member variable _type

ScalarConvectionDiffusion:
	 {'diffusionCoefficient': 0.5, 'sourceFieldName': 'R', 'scalarFieldName': 'T', 'velocityFieldName': 'U'}


In [65]:
# changing boundary conditions
myFields = sim1.getFields()
T = myFields['T']
T.setBoundaryCondition('top', 'zeroGradient')
T.setBoundaryCondition('bottom', 'zeroGradient')
T.setBoundaryCondition('left', T_l)
T.setBoundaryCondition('right', T_r)

# setting conductivity and heat source:
#myFields['k'] = k
R = myFields['R']
R._raw[:,:] = S

sim1.showfields()

sim1.compile()

T.solve()

variable fields :
	 T 	 variableCellField 	 {'left': 100, 'right': 200, 'top': 'zeroGradient', 'bottom': 'zeroGradient'}
parameter fields :
	 U 	 parameterFaceField
	 R 	 scalar
	 k 	 scalar




ValueError: operands could not be broadcast together with shapes (10,47) (10,49) (10,47) 

In [ ]:
# post-processing:
import numpy as np
import matplotlib.pyplot as plt

sim1.display(T)

nbcellsX = sim1._mesh._cells_x
cellSpacing = sim1._mesh._uniformSpacing
L = geom._lenX

xSim = np.linspace(0+0.5*cellSpacing,LenX-0.5*cellSpacing,nbcellsX)
ySim = T._raw[0,:]

xTheo = np.linspace(0,LenX, 100)
yTheor = np.array([yFunc(x) for x in xTheo])

ax = plt.gca()
ax.plot(xSim, ySim, 'x', label='simulation')
ax.plot(xTheo, yTheor, label='theoretical')

plt.legend()
plt.show()

print(ySim)